The data is not complicate, so for the assumption, I respect the accurancy of the data I get from morse and the weather report website. What I should do with ETL is mostly to transform for further use.

The the beginning, I did ETL with general coding, such web crawling, backfill and transform.
When it comes to analysis, I used Spark Pipeline for one hot encoding and assembling feature.

# Morse Data Load

In [1]:
import pandas as pd

In [2]:
morse=pd.read_csv('morse.csv')

In [3]:
morse.head(10)

,local_created_at,item_name,net_quantity
0,04/22/2016 16:03:07,Latte,1
1,09/20/2016 09:24:23,Latte,1
2,12/27/2016 13:09:00,"Cappuccino, Unknown",1
3,10/03/2016 10:17:46,New Orleans Iced Coffee (cup),2
4,09/17/2016 14:33:18,Sparkling Water,1
5,07/07/2016 08:06:00,Chemex Coffee,1
6,03/17/2016 08:15:40,Drip Coffee,2
7,09/09/2016 11:11:00,"Cappuccino, Unknown",1
8,02/23/2016 11:05:25,New Orleans Iced Coffee (cup),1
9,03/27/2016 09:57:48,Apple Juice,1


### Data Modeling

The first step is the collection data of temperature in 2016, making it ready to use for further analysis.

Then merge weather data with the original data, make it clean for well-formatted for analysis, such as the date-formatting of the create_time. The temperature has to the in one day, so transform the "local_created_at" should be necessary, or the two table cannot be merged.

Due to the comparably small dataset, EC2 is not necessary. I first used Pandas SQL for the questions, which can be copied and run in any computer. However, because of its terrible apparence, I turned to Spark. I used Spark SQL for the questions again, with different queries, because their languages are slightly different from each other.

Per the two questions, I understand you are interested in the relationship of sales and weather. I used Spark Linear Regression to do prediction with item_name and temperature, and it shows the independent variables don't work well in the model. I think the assumption should have worked, because it is logical. The reason of the conclusion is the weather pattern in Oakland. In other words, it doesn't change dramastically all the time, so slight difference can be reasonably ingored by people.

# Temperature Crawling

2016 Oakland Weather History
https://www.usclimatedata.com/climate/oakland/california/united-states/usca2500/2016/1

In [4]:
from lxml import html
import requests
import pandas as pd

In [46]:
days=[]
for i in range(1,13):
    web='https://www.usclimatedata.com/climate/oakland/california/united-states/usca2500/2016/'+str(i)
    page = requests.get(web)
    tree = html.fromstring(page.content)
    day=tree.xpath('//td[@class="align_left daily_climate_table_td_day"]/text()')
    days.extend(day[31:])

In [90]:
days

['1 jan 2016',
 '2 jan 2016',
 '3 jan 2016',
 '4 jan 2016',
 '5 jan 2016',
 '6 jan 2016',
 '7 jan 2016',
 '8 jan 2016',
 '9 jan 2016',
 '10 jan 2016',
 '11 jan 2016',
 '12 jan 2016',
 '13 jan 2016',
 '14 jan 2016',
 '15 jan 2016',
 '16 jan 2016',
 '17 jan 2016',
 '18 jan 2016',
 '19 jan 2016',
 '20 jan 2016',
 '21 jan 2016',
 '22 jan 2016',
 '23 jan 2016',
 '24 jan 2016',
 '25 jan 2016',
 '26 jan 2016',
 '27 jan 2016',
 '28 jan 2016',
 '29 jan 2016',
 '30 jan 2016',
 '31 jan 2016',
 '1 feb 2016',
 '2 feb 2016',
 '3 feb 2016',
 '4 feb 2016',
 '5 feb 2016',
 '6 feb 2016',
 '7 feb 2016',
 '8 feb 2016',
 '9 feb 2016',
 '10 feb 2016',
 '11 feb 2016',
 '12 feb 2016',
 '13 feb 2016',
 '14 feb 2016',
 '15 feb 2016',
 '16 feb 2016',
 '17 feb 2016',
 '18 feb 2016',
 '19 feb 2016',
 '20 feb 2016',
 '21 feb 2016',
 '22 feb 2016',
 '23 feb 2016',
 '24 feb 2016',
 '25 feb 2016',
 '26 feb 2016',
 '27 feb 2016',
 '28 feb 2016',
 '1 mar 2016',
 '2 mar 2016',
 '3 mar 2016',
 '4 mar 2016',
 '5 mar 2016',

format date

temperature on 2016-02-29 was missing in the website

In [109]:
from datetime import date, timedelta

d1 = date(2016, 1, 1)  # start date
d2 = date(2016, 12, 31)  # end date

delta = d2 - d1         # timedelta

dates=[]
for i in range(delta.days + 1):
    date=d1 + timedelta(i)
    dates.append(date)
    #print(date)

In [111]:
len(dates)

366

In [70]:
highs=[]
for i in range(1,13):
    web='https://www.usclimatedata.com/climate/oakland/california/united-states/usca2500/2016/'+str(i)
    page = requests.get(web)
    tree = html.fromstring(page.content)
    high =tree.xpath('//td[@class="align_right climate_table_data_td temperature_red "]/text()')
    highs.extend(high[31:])

### Data backfill
The temperatere of close days should be like each other

In [73]:
# replace null values with the closest day's temperatere
def replace(lst):
    curr=lst[0]
    for i in range(len(lst)):
        if lst[i]=='-':
            lst[i]=curr
        else:
            curr=lst[i]

In [74]:
replace(highs)

In [168]:
highest=highs[:59]+[highs[59]]+highs[59:]

In [191]:
def to_int(lst):
    for i in range(len(lst)):
        a=lst[i].split('.')
        lst[i]=int(a[0])

In [192]:
to_int(highest)

In [53]:
lows=[]
for i in range(1,13):
    web='https://www.usclimatedata.com/climate/oakland/california/united-states/usca2500/2016/'+str(i)
    page = requests.get(web)
    tree = html.fromstring(page.content)
    low =tree.xpath('//td[@class="align_right climate_table_data_td temperature_blue"]/text()')
    lows.extend(low[31:])

In [77]:
replace(lows)

In [169]:
lowest=lows[:59]+[lows[59]]+lows[59:]

In [194]:
to_int(lowest)

In [195]:
temperature=pd.DataFrame(dates,columns=['date'])
temperature['High'] = pd.Series(highest, index=temperature.index)
temperature['Low'] = pd.Series(lowest, index=temperature.index)

In [196]:
temperature.head(5)

,date,High,Low
0,2016-01-01,55,39
1,2016-01-02,55,44
2,2016-01-03,62,46
3,2016-01-04,62,46
4,2016-01-05,57,48


# Merge Temperature with Original Morse Date

In [133]:
import datetime
morse['date'] = morse['local_created_at'].dt.date

In [197]:
morse_tmp = pd.merge(morse, temperature, on=['date'])

In [198]:
morse_tmp.head(10)

,local_created_at,item_name,net_quantity,date,High,Low
0,2016-04-22 16:03:07,Latte,1,2016-04-22,63,55
1,2016-04-22 10:54:16,Drip Coffee,1,2016-04-22,63,55
2,2016-04-22 14:05:23,Gibraltar,1,2016-04-22,63,55
3,2016-04-22 08:15:55,Latte,1,2016-04-22,63,55
4,2016-04-22 16:36:54,New Orleans Iced Coffee (cup),1,2016-04-22,63,55
5,2016-04-22 07:57:24,Drip Coffee,1,2016-04-22,63,55
6,2016-04-22 08:23:28,Drip Coffee,2,2016-04-22,63,55
7,2016-04-22 07:58:55,Latte,1,2016-04-22,63,55
8,2016-04-22 15:32:21,Apple Juice,1,2016-04-22,63,55
9,2016-04-22 11:37:48,Drip Coffee,1,2016-04-22,63,55


In [201]:
morse_tmp['Average'] = morse_tmp.apply(lambda row: int((row.High + row.Low)/2), axis=1)

In [203]:
morse_tmp.head(5)

,local_created_at,item_name,net_quantity,date,High,Low,Average
0,2016-04-22 16:03:07,Latte,1,2016-04-22,63,55,59
1,2016-04-22 10:54:16,Drip Coffee,1,2016-04-22,63,55,59
2,2016-04-22 14:05:23,Gibraltar,1,2016-04-22,63,55,59
3,2016-04-22 08:15:55,Latte,1,2016-04-22,63,55,59
4,2016-04-22 16:36:54,New Orleans Iced Coffee (cup),1,2016-04-22,63,55,59


In [265]:
morse_tmp.to_csv('morse_tmp.csv')

In [135]:
import pandas as pd
morse_new=pd.read_csv('morse_tmp.csv')

In [141]:
df=morse_new.groupby('date').agg({'net_quantity':'sum', 'High':'mean', 'Low':'mean', 'Average':'mean'})

In [143]:
df.to_csv('morse_sum.csv')

# SQL Analysis

## Data Partioning

It will be more about schema design. However, the dataset is not that large that worth different tables for data partioning. But as far as I am concerned, I'd like to design, Item table(with item_names), Transaction table (create_time, item_name, new_quantity, date), Weather table (with date highest, lowest, average temperature).

The main limitation is the values in net_quantity col, most of those are 1 or 2, makeing it inefficient to seperate to different tables, and it will cost more time to select and join.

In [2]:
import pandas as pd
morse_tmp=pd.read_csv('morse_tmp.csv')

In [3]:
from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False)

In [4]:
morse_tmp.to_sql('morse_tmp', con=engine)

In [5]:
engine.execute("SELECT * FROM morse_tmp LIMIT 5").fetchall()

[(0, 0, '2016-04-22 16:03:07', 'Latte', 1, '2016-04-22', 63, 55, 59),
 (1, 1, '2016-04-22 10:54:16', 'Drip Coffee', 1, '2016-04-22', 63, 55, 59),
 (2, 2, '2016-04-22 14:05:23', 'Gibraltar', 1, '2016-04-22', 63, 55, 59),
 (3, 3, '2016-04-22 08:15:55', 'Latte', 1, '2016-04-22', 63, 55, 59),
 (4, 4, '2016-04-22 16:36:54', 'New Orleans Iced Coffee (cup)', 1, '2016-04-22', 63, 55, 59)]

In [205]:
from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False)

In [255]:
morse_tmp.to_sql('morse_tmp', con=engine)

In [256]:
engine.execute("SELECT * FROM morse_tmp LIMIT 5").fetchall()

[(0, '2016-04-22 16:03:07.000000', 'Latte', 1, '2016-04-22', 63, 55, 59),
 (1, '2016-04-22 10:54:16.000000', 'Drip Coffee', 1, '2016-04-22', 63, 55, 59),
 (2, '2016-04-22 14:05:23.000000', 'Gibraltar', 1, '2016-04-22', 63, 55, 59),
 (3, '2016-04-22 08:15:55.000000', 'Latte', 1, '2016-04-22', 63, 55, 59),
 (4, '2016-04-22 16:36:54.000000', 'New Orleans Iced Coffee (cup)', 1, '2016-04-22', 63, 55, 59)]

## First question
Write a SQL query to generate the report of best-sellers in the following form

In [298]:
engine.execute("SELECT tmp as Temperature, item_name, MAX(total) as number_sold FROM (SELECT AVG(Average) as tmp, item_name, sum(net_quantity) as total FROM morse_tmp GROUP BY item_name) temp").fetchall()

[(60.49268018018018, 'Latte', 1945)]

## Second question
Write a SQL query to generate the average change on the number of items sold when the temperature is
colder by 2 degrees Farenheit or warmer by 2 degrees Farenheit the next day.

In [7]:
engine.execute("SELECt c.item_name, warm, cold from (SELECT a.item_name,AVG(a.net_quantity-b.net_quantity) as warm from morse_tmp a JOIN morse_tmp b ON JulianDay(a.date)-JulianDay(b.date)=1 WHERE a.Average-b.Average=-2 GROUP BY a.item_name) c join (SELECT m.item_name,AVG(m.net_quantity-n.net_quantity) as cold from morse_tmp m JOIN morse_tmp n ON JulianDay(m.date)-JulianDay(n.date)=1 WHERE m.Average-n.Average=-2 GROUP BY m.item_name) d on c.item_name=d.item_name").fetchall()

[('Americano', -0.10726351351351351, -0.10726351351351351),
 ('Apple Juice', -0.08775981524249422, -0.08775981524249422),
 ('Au Lait, Unknown', -0.1033210332103321, -0.1033210332103321),
 ('Cappuccino, Unknown', -0.013879408418657566, -0.013879408418657566),
 ('Cascara Fizz', -0.08882521489971347, -0.08882521489971347),
 ('Chemex Coffee', -0.025252525252525252, -0.025252525252525252),
 ('Drip Coffee', 0.02336754749945403, 0.02336754749945403),
 ('Espresso', -0.1291053227633069, -0.1291053227633069),
 ('Gibraltar', -0.11982881597717546, -0.11982881597717546),
 ('Hot Chocolate', -0.034482758620689655, -0.034482758620689655),
 ('Latte', -0.045156156753530935, -0.045156156753530935),
 ('Macchiato', 0.20679012345679013, 0.20679012345679013),
 ('Milk', -0.009615384615384616, -0.009615384615384616),
 ('Mocha', -0.05569620253164557, -0.05569620253164557),
 ('NOLA Carton', 0.5806451612903226, 0.5806451612903226),
 ('New Orleans Iced Coffee (cup)', 0.1372634326734827, 0.1372634326734827),
 ('S.O

# Spark Analysis

In [320]:
spark.stop()

In [9]:
import findspark
findspark.init()
from pyspark.sql import SQLContext
from pyspark import SparkContext
from pyspark.sql.types import *
sc =SparkContext()
sqlContext = SQLContext(sc)

In [13]:
df = sqlContext.read.load('morse_tmp.csv', format='com.databricks.spark.csv', header='true', inferSchema='true')
df.show(10)

+---+--------------------+--------------------+------------+--------------------+----+---+-------+
|_c0|    local_created_at|           item_name|net_quantity|                date|High|Low|Average|
+---+--------------------+--------------------+------------+--------------------+----+---+-------+
|  0|2016-04-22 16:03:...|               Latte|           1|2016-04-22 00:00:...|  63| 55|     59|
|  1|2016-04-22 10:54:...|         Drip Coffee|           1|2016-04-22 00:00:...|  63| 55|     59|
|  2|2016-04-22 14:05:...|           Gibraltar|           1|2016-04-22 00:00:...|  63| 55|     59|
|  3|2016-04-22 08:15:...|               Latte|           1|2016-04-22 00:00:...|  63| 55|     59|
|  4|2016-04-22 16:36:...|New Orleans Iced ...|           1|2016-04-22 00:00:...|  63| 55|     59|
|  5|2016-04-22 07:57:...|         Drip Coffee|           1|2016-04-22 00:00:...|  63| 55|     59|
|  6|2016-04-22 08:23:...|         Drip Coffee|           2|2016-04-22 00:00:...|  63| 55|     59|
|  7|2016-

In [16]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- local_created_at: timestamp (nullable = true)
 |-- item_name: string (nullable = true)
 |-- net_quantity: integer (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- High: integer (nullable = true)
 |-- Low: integer (nullable = true)
 |-- Average: integer (nullable = true)



In [17]:
df.createOrReplaceTempView("morse_tmp")

In [19]:
sqlContext.sql("SELECT * FROM morse_tmp LIMIT 5").show()

+---+--------------------+--------------------+------------+--------------------+----+---+-------+
|_c0|    local_created_at|           item_name|net_quantity|                date|High|Low|Average|
+---+--------------------+--------------------+------------+--------------------+----+---+-------+
|  0|2016-04-22 16:03:...|               Latte|           1|2016-04-22 00:00:...|  63| 55|     59|
|  1|2016-04-22 10:54:...|         Drip Coffee|           1|2016-04-22 00:00:...|  63| 55|     59|
|  2|2016-04-22 14:05:...|           Gibraltar|           1|2016-04-22 00:00:...|  63| 55|     59|
|  3|2016-04-22 08:15:...|               Latte|           1|2016-04-22 00:00:...|  63| 55|     59|
|  4|2016-04-22 16:36:...|New Orleans Iced ...|           1|2016-04-22 00:00:...|  63| 55|     59|
+---+--------------------+--------------------+------------+--------------------+----+---+-------+



## First question
Write a SQL query to generate the report of best-sellers in the following form

In [36]:
sqlContext.sql("SELECT tmp as Temperature, item_name, total as number_sold FROM (SELECT AVG(Average) as tmp, item_name, sum(net_quantity) as total FROM morse_tmp GROUP BY item_name) temp ORDER BY total DESC LIMIT 1").show()



+-----------------+---------+-----------+
|      Temperature|item_name|number_sold|
+-----------------+---------+-----------+
|60.49268018018018|    Latte|       1945|
+-----------------+---------+-----------+



## Second question
Write a SQL query to generate the average change on the number of items sold when the temperature is
colder by 2 degrees Farenheit or warmer by 2 degrees Farenheit the next day.

In [22]:
sqlContext.sql("SELECt c.item_name, warm, cold from (SELECT a.item_name,AVG(a.net_quantity-b.net_quantity) as warm from morse_tmp a JOIN morse_tmp b ON DATEDIFF(a.date,b.date)=1 WHERE a.Average-b.Average=-2 GROUP BY a.item_name) c join (SELECT m.item_name,AVG(m.net_quantity-n.net_quantity) as cold from morse_tmp m JOIN morse_tmp n ON DATEDIFF(m.date,n.date)=1 WHERE m.Average-n.Average=-2 GROUP BY m.item_name) d on c.item_name=d.item_name").show()

+--------------------+--------------------+--------------------+
|           item_name|                warm|                cold|
+--------------------+--------------------+--------------------+
|         Apple Juice|-0.08775981524249422|-0.08775981524249422|
|            Espresso| -0.1291053227633069| -0.1291053227633069|
|                 Tea|-0.12535612535612536|-0.12535612535612536|
|     Sparkling Water|-0.10875331564986737|-0.10875331564986737|
|                Milk|-0.00961538461538...|-0.00961538461538...|
|           Shakerato|-0.04545454545454...|-0.04545454545454...|
|S.O. Iced - 4 oz Oji|-0.09822866344605476|-0.09822866344605476|
|         Still Water|-0.10869565217391304|-0.10869565217391304|
|               Mocha|-0.05569620253164557|-0.05569620253164557|
|               Latte|-0.04515615675353...|-0.04515615675353...|
|           Americano|-0.10726351351351351|-0.10726351351351351|
|    Au Lait, Unknown| -0.1033210332103321| -0.1033210332103321|
|           Gibraltar|-0.

## Analysis with ML

In [37]:
print("Total number of rows: %d" % df.count())

Total number of rows: 10000


### Seperate with different products

In [88]:
sqlContext.sql("SELECT count(DISTINCT item_name) FROM morse_tmp").show()

+-------------------------+
|count(DISTINCT item_name)|
+-------------------------+
|                       25|
+-------------------------+



In [63]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, RegexTokenizer
from pyspark.ml.feature import VectorAssembler

In [89]:
regexTokenizer_name = RegexTokenizer(inputCol="item_name", outputCol="name", pattern="\\W")
hashingTF_name = HashingTF(inputCol="name", outputCol="rawName", numFeatures=25)
assembler = VectorAssembler(inputCols=['rawName', 'High', 'Low', 'Average'],outputCol='features')
pipeline = Pipeline(stages=[regexTokenizer_name,hashingTF_name,assembler])
pipelineFit = pipeline.fit(df)
Morse = pipelineFit.transform(df)
Morse=Morse.select(['features', 'net_quantity'])
Morse.show(5)

+--------------------+------------+
|            features|net_quantity|
+--------------------+------------+
|(28,[19,25,26,27]...|           1|
|(28,[1,2,25,26,27...|           1|
|(28,[23,25,26,27]...|           1|
|(28,[19,25,26,27]...|           1|
|(28,[0,1,11,19,25...|           1|
+--------------------+------------+
only showing top 5 rows



In [90]:
(training_data, test_data) = Morse.randomSplit([0.7, 0.3], seed = 100)
print("Training Dataset Count: " + str(trainingData.count()))
print("Test Dataset Count: " + str(testData.count()))

Training Dataset Count: 6047
Test Dataset Count: 3953


In [91]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol = 'features', labelCol='net_quantity', maxIter=10, regParam=0.3, elasticNetParam=0.8)
lr_model = lr.fit(training_data)
print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))

Coefficients: [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
Intercept: 1.108476501490842


In [92]:
trainingSummary = lr_model.summary
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

RMSE: 0.380768
r2: -0.000000


The result is reliable to say, it cannot be a useful prediction. 

## Predict with Single day features, getting rid of item_name

In [162]:
sqlContext.sql("SELECT SUM(net_quantity) as net_quantity, AVG(High) as High, AVG(Low) as Low, AVG(Average) as Average, Date FROM morse_tmp Group by date limit 10").show()

+------------+----+----+-------+--------------------+
|net_quantity|High| Low|Average|                Date|
+------------+----+----+-------+--------------------+
|          29|80.0|57.0|   68.0|2016-06-20 00:00:...|
|          30|61.0|51.0|   56.0|2016-12-10 00:00:...|
|          55|62.0|46.0|   54.0|2016-11-23 00:00:...|
|          36|70.0|55.0|   62.0|2016-08-16 00:00:...|
|          41|72.0|57.0|   64.0|2016-11-11 00:00:...|
|          30|75.0|59.0|   67.0|2016-09-02 00:00:...|
|          23|66.0|54.0|   60.0|2016-04-11 00:00:...|
|          26|66.0|53.0|   59.0|2016-05-12 00:00:...|
|          26|80.0|50.0|   65.0|2016-03-17 00:00:...|
|          27|70.0|55.0|   62.0|2016-05-29 00:00:...|
+------------+----+----+-------+--------------------+



### Above is what it will be for prediction

In [148]:
morse_sum = sqlContext.read.load('morse_sum.csv', format='com.databricks.spark.csv', header='true', inferSchema='true')
morse_sum.show(10)

+--------------------+------------+----+---+-------+
|                date|net_quantity|High|Low|Average|
+--------------------+------------+----+---+-------+
|2016-01-23 00:00:...|           1|  61| 52|     56|
|2016-01-24 00:00:...|           4|  62| 46|     54|
|2016-02-04 00:00:...|           1|  63| 46|     54|
|2016-02-05 00:00:...|           1|  70| 43|     56|
|2016-02-06 00:00:...|          25|  70| 46|     58|
|2016-02-07 00:00:...|          46|  73| 46|     59|
|2016-02-08 00:00:...|          52|  81| 55|     68|
|2016-02-09 00:00:...|          44|  73| 48|     60|
|2016-02-10 00:00:...|          35|  71| 51|     61|
|2016-02-11 00:00:...|          17|  73| 51|     62|
+--------------------+------------+----+---+-------+
only showing top 10 rows



In [149]:
assembler = VectorAssembler(inputCols=['High', 'Low', 'Average'],outputCol='features')
pipeline = Pipeline(stages=[assembler])
pipelineFit = pipeline.fit(morse_sum)
clean_Morse = pipelineFit.transform(morse_sum)
clean_Morse=clean_Morse.select(['features', 'net_quantity'])
clean_Morse.show(10)

+----------------+------------+
|        features|net_quantity|
+----------------+------------+
|[61.0,52.0,56.0]|           1|
|[62.0,46.0,54.0]|           4|
|[63.0,46.0,54.0]|           1|
|[70.0,43.0,56.0]|           1|
|[70.0,46.0,58.0]|          25|
|[73.0,46.0,59.0]|          46|
|[81.0,55.0,68.0]|          52|
|[73.0,48.0,60.0]|          44|
|[71.0,51.0,61.0]|          35|
|[73.0,51.0,62.0]|          17|
+----------------+------------+
only showing top 10 rows



In [155]:
(training, test) = clean_Morse.randomSplit([0.7, 0.3], seed = 100)
print("Training Dataset Count: " + str(training.count()))
print("Test Dataset Count: " + str(test.count()))

Training Dataset Count: 232
Test Dataset Count: 99


In [156]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol = 'features', labelCol='net_quantity', maxIter=10, regParam=0.3, elasticNetParam=0.8)
lr_model = lr.fit(training)
print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))

Coefficients: [0.0,-0.041752701873271444,0.0]
Intercept: 35.48766655679977


In [159]:
lr_predictions = lr_model.transform(test)
lr_predictions.select("prediction","net_quantity","features").show(5)
from pyspark.ml.evaluation import RegressionEvaluator
lr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="net_quantity",metricName="r2")
print("R Squared (R2) on test data = %g" % lr_evaluator.evaluate(lr_predictions))

+-----------------+------------+----------------+
|       prediction|net_quantity|        features|
+-----------------+------------+----------------+
|33.94281658748873|          24|[55.0,37.0,46.0]|
|33.65054767437583|          38|[55.0,44.0,49.0]|
|33.56704227062929|          20|[55.0,46.0,50.0]|
| 33.4000314631362|          32|[55.0,50.0,52.0]|
| 33.4000314631362|          30|[57.0,50.0,53.0]|
+-----------------+------------+----------------+
only showing top 5 rows

R Squared (R2) on test data = 0.000669191


In [157]:
trainingSummary = lr_model.summary
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

RMSE: 11.358334
r2: 0.001003


In [160]:
test_result = lr_model.evaluate(test)
print("Root Mean Squared Error (RMSE) on test data = %g" % test_result.rootMeanSquaredError)

Root Mean Squared Error (RMSE) on test data = 10.8016


### Both predictions looks far from decent conclusion
It probably indicates that the temperature should not be a good feature in the model. By all means, the weather doesn't change dramastically in Oakland CA. So the assumption might work in the east coast.